In [2]:
import os
os.chdir("drive/MyDrive/CapDec")

In [2]:
!ls

annotations		 figures	      parse_karpathy.py       README.md
coco_train		 flickr8kforStyle     post_processed_karpthy  requirments.txt
custom_types.py		 gpt2_prefix_eval.py  predictions_runner.py   train.ipynb
data			 gpt2_prefix.py       __pycache__	      train.py
embeddings_generator.py  others		      pycocoevalcap	      transformer_mapper.py


In [ ]:
!python parse_karpathy.py

In [9]:
#@title Install
!pip install transformers
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-06ufrxw8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-06ufrxw8
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 757.6 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=f17856cbf89c8d2c18817b403f291536d914886b992be8213efbbd6fa2b6dbc4
  Stored in directory: /tmp/pip-ephem-wheel-cache-wypnvo5j/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
!python embeddings_generator.py  --dataset_mode 1

in main
in main
in main2
in main3
**dataset Namespace(clip_model_type='RN50x4', dataset_mode=1.0, fix_gender_imbalance_mode=0)
Text embeddings will be added to the dataset
flickr8kforStyle/Images
out_path is ./data/flicker30_RN50x4_train_with_text_embeddings_not_norm.pkl fix gender imbalance is 0
30000 captions loaded from json 
100% 30000/30000 [05:43<00:00, 87.32it/s]
Done
long_caps bigger then 76 amount was = 0
30000 embeddings saved 
not found images = 0
text embeddings = True


In [ ]:
!python train.py --data FLICKR --out_dir ./coco_train/ --noise_variance 0.016

data FLICKR
./data/flicker30_RN50x4_train_with_text_embeddings_not_norm.pkl

vocab.json: 100% 1.04M/1.04M [00:00<00:00, 43.5MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 1.93MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 5.27MB/s]
config.json: 100% 665/665 [00:00<00:00, 3.85MB/s]
Data size is 30000
model.safetensors: 100% 548M/548M [00:02<00:00, 262MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 640kB/s]
Train both prefix and GPT
modality_offset=False
args saved to file ./coco_train//train_commandline_args.txt
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
>>> Training epoch 0 / 10
coco_prefix: 100% 1875/1875 [15:07<00:00,  2.07it/s, loss=2.38]
loss_per_epoch_train:  [3.6324902154922487]
>>> Training epoch 1 / 10

In [ ]:
!python predictions_runner.py  --checkpoint coco_train/coco_prefix-009.pt --dataset_mode 1


start....
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 15.5MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 286MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 46.8MB/s]
config.json: 100% 665/665 [00:00<00:00, 3.66MB/s]
loaded tokenizer
beam search = True
loaded data
<class 'list'>
5000
sample example:  {'image_id': 2090545563, 'caption': 'the boy laying face down on a skateboard is being pushed along the ground by another boy .', 'id': 30000, 'filename': '2090545563_a4e66ec76b.jpg'}
out_path = coco_train/coco_prefix-009.json, dataset_mode = 1
args saved to file coco_train/pred_commandline_args.txt
model.safetensors: 100% 548M/548M [00:05<00:00, 98.7MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 749kB/s]
coco_train/coco_prefix-009.pt
modality_offset=False
100%|███████████████████████████████████████| 402M/402M [00:05<00:00, 81.4MiB/s]
mean: 8912.72 ms, std: 0.00 ms
(2090545563, 'the boy laying face down on a skateboard is being pushed along the ground by another boy .', 'two 

In [ ]:
!git clone https://github.com/sks3i/pycocoevalcap.git

Cloning into 'pycocoevalcap'...
remote: Enumerating objects: 30, done.
remote: Total 30 (delta 0), reused 0 (delta 0), pack-reused 30
Receiving objects: 100% (30/30), 70.15 MiB | 15.09 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (20/20), done.


In [6]:
import json
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

class COCOEvalCap:
    def __init__(self, coco_path, cocoRes_path):
        with open(coco_path, 'r') as f:
            self.coco = json.load(f)

        with open(cocoRes_path, 'r') as f:
            self.cocoRes = json.load(f)

        self.evalImgs = []
        self.eval = {}
        self.imgToEval = {}
        self.params = {'image_id': [entry['image_id'] for entry in self.cocoRes]}

    def find_first_caption_by_id(self, image_id, captions):
        for caption_entry in captions:
            if caption_entry['image_id'] == image_id:
                return caption_entry['caption']
        return None

    def evaluate(self):
        imgIds = self.params['image_id']

        gts = {entry['image_id']: [caption['caption'] for caption in self.coco if caption['image_id'] == entry['image_id']] for entry in self.cocoRes}
        res = {entry['image_id']: [caption['caption'] for caption in self.cocoRes if caption['image_id'] == entry['image_id']] for entry in self.cocoRes}
        print(res)
        print(gts)


        # print([print(c) for k, v in res.items() for c in v])
        # sentences = '\n'.join([c.replace('\n', ' ') for k, v in res.items() for c in v])
        # sentences = '\n'.join([c.replace('\n', ' ') for k, v in gts.items() for c in v])
        # print(sentences)

        # gts = {entry['image_id']: entry['caption'] for entry in self.coco}
        # res = {entry['image_id']: entry['caption'] for entry in self.cocoRes}

        # Tokenization
        print(len(res.keys()))
        print(len(gts.keys()))
        print('tokenization...')
        tokenizer = PTBTokenizer()
        gts = tokenizer.tokenize(gts)
        res = tokenizer.tokenize(res)
        print(len(res.keys()))
        print(len(gts.keys()))

        # Set up scorers
        print('setting up scorers...')
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(), "METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr")
        ]

        # Compute scores
        eval = {}
        for scorer, method in scorers:
            print('computing %s score...' % (scorer.method()))
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, imgIds, m)
                    print("%s: %0.3f" % (m, sc))
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, imgIds, method)
                print("%s: %0.3f" % (method, score))
        self.setEvalImgs()

    def setEval(self, score, method):
        self.eval[method] = score

    def setImgToEvalImgs(self, scores, imgIds, method):
        for imgId, score in zip(imgIds, scores):
            if imgId not in self.imgToEval:
                self.imgToEval[imgId] = {}
                self.imgToEval[imgId]["image_id"] = imgId
            self.imgToEval[imgId][method] = score

    def setEvalImgs(self):
        self.evalImgs = [eval for imgId, eval in self.imgToEval.items()]

# Example usage
coco_path = 'post_processed_karpthy/val.json'
cocoRes_path = 'coco_train/coco_prefix-004.json'
evaluator = COCOEvalCap(coco_path, cocoRes_path)
evaluator.evaluate()


{2090545563: ['two children play with a peace sign on a subway.........................................................', 'two children play with a peace sign on a subway.........................................................', 'two children play with a peace sign on a subway.........................................................', 'two children play with a peace sign on a subway.........................................................', 'two children play with a peace sign on a subway.........................................................'], 3393035454: ['an african-american african-american african-american african-american african-american african-american african-american african-american african-american african-american african-american african-american african-american af', 'an african-american african-american african-american african-american african-american african-american african-american african-american african-american african-american african-american african-ame